In [1]:
cd(@__DIR__)
ENV["CELLLISTMAP_8.3_WARNING"] = "false"
include("../src/juliaEAM.jl")

using Pkg
Pkg.activate(".")

using Printf
using AtomsCalculators
using ASEconvert # use this PR:https://github.com/mfherbst/ASEconvert.jl/pull/17, Pkg.add(url="https://github.com/tjjarvinen/ASEconvert.jl.git", rev="atomscalculators")
using Unitful: Å, nm
using PythonCall
ENV["PYTHON"] = "/SNS/users/ccu/miniconda3/envs/analysis/bin/python"
# install the following packages in julia REPL
# using CondaPkg
# CondaPkg.add_pip("IPython")
# CondaPkg.add_pip("nglview")
using StaticArrays: SVector
using GLMakie
using Molly
using Zygote
using LinearAlgebra
import Interpolations:cubic_spline_interpolation, linear_interpolation, interpolate, BSpline, Cubic, scale, Line, OnGrid, extrapolate, Gridded, extrapolate, Flat
using DelimitedFiles
using UnitfulAtomic
import PeriodicTable

  Activating new project at `~/Documents/ABCD_J/EAM`


In [2]:
function repeat(fun,times)
    for i in 1:times
        fun()
    end
end

repeat (generic function with 1 method)

## Define a Molly system wo interaction 

In [3]:
## 1. Import ASE and other Python modules
# Import ASE and other Python modules as needed
ase = pyimport("ase")
ase_view = pyimport("ase.visualize")

al_LatConst = 4.0495/10 # nm
atom_mass = 26.9815u"u"  # Atomic mass of aluminum in grams per mole
function system_adatom(systemsize)

    # Build an (001) Al surface  
    # atoms_ase = ase.build.fcc100("Al", size=size, vacuum = al_LatConst*4*10)
    Albulk = ase.build.bulk("Al", "fcc", a=al_LatConst*10, cubic=true)
    atoms_ase = ase.build.surface(Albulk, (0,0,1), Int(systemsize[3]/2), vacuum = al_LatConst*4*10)
    current_atoms = atoms_ase
    for i in 1:systemsize[1]
        if i<2
            continue
        end
        current_atoms = ase.build.stack(current_atoms, atoms_ase, axis=0)
    end
    current_atoms_i = current_atoms
    for j in 1:systemsize[2]
        if j<2
            continue
        end
        current_atoms = ase.build.stack(current_atoms, current_atoms_i, axis=1)
    end
    # The basis vectors on x and y are along 1/2<110> directions
    ase.build.add_adsorbate(current_atoms, "Al", al_LatConst*10/2, position=(al_LatConst*10*4.0/2,al_LatConst*10*4.0/2))
    # ase.build.add_adsorbate(atoms_ase, "Al", al_LatConst/2, "bridge")

    atoms_ase = current_atoms
    atoms_ase.translate([al_LatConst*10/4,al_LatConst*10/4,0])
    atoms_ase.wrap()

    atoms_ase_cell = atoms_ase.get_cell()
    box_size = pyconvert(Array{Float64}, [atoms_ase_cell[x,x] for x in range(0,2)])*u"Å"
    box_size[1]*=0.999 # distort the box size slightly
    box_size[2]*=1.001 # distort the box size slightly

    # Build an Julia AtomsBase abstract 
    atoms_ab = pyconvert(AbstractSystem, atoms_ase)

    ## 4. Create Molly system
    ### 4.1 Convert atom positions to Molly's expected format (nanometers) and create Molly.Atom objects
    # Get atom positions from previously defined ASE system
    function get_positions(atoms_ase)
        positions = [(atom.position[1], atom.position[2], atom.position[3]) for atom in atoms_ase]
        return positions
    end

    # Convert each position from Ångströms to nanometers and ensure the conversion is applied element-wise.
    atom_positions = [SVector(uconvert(Å, pos[1]), 
        uconvert(Å, pos[2]), uconvert(Å, pos[3])) for pos in get_positions(atoms_ab)]

    molly_atoms = [Molly.Atom(index=i, charge=0, mass=atom_mass, 
                            #   σ=2.0u"Å" |> x -> uconvert(u"nm", x), ϵ=ϵ_kJ_per_mol
                            ) for i in 1:length(atom_positions)]
    return molly_atoms, atoms_ab, box_size, atom_positions, atoms_ase
end

system_adatom (generic function with 1 method)

## Incoporate into `AtomCalculators` force/energy calculator and `Molly` simulator

In [4]:
eam = EAM()
fname = "Al99.eam.alloy"
read_potential!(eam, fname)

### Define customized interaction type in `AtomsCalculators`

In [5]:
struct EAMInteractionJulia
    calculator::Any  # Holds the ASE EAM calculator reference
    f_energy::Any    # Holds the energy function
    f_forces::Any    # Holds the forces function
end

### Define `Molly` style ABCSimulator

In [6]:
# Define the ABCSimulator structure
"""
In the constructor function ABCSimulator, default values are provided for each of these fields. 
If you create a SteepestDescentMinimizer without specifying the types, default values 
will determine the types of the fields. For example, if you create a ABCSimulator without specifying sigma, 
it will default to 0.1*u"Å", and S will be the type of this value.
"""
struct ABCSimulator{S,W,D,F,L}
    sigma::S 
    W::W
    max_steps::Int
    max_steps_minimize::Int
    step_size_minimize::D
    tol::F
    log_stream::L
end

"""
ABCSimulator(; sigma=0.1*u"Å", W=1e-2*u"eV", max_steps=100, max_steps_minimize=100, step_size_minimize=0.1u"Å", tol=1e-4u"eV/Å", log_stream=devnull)

Constructor for ABCSimulator.

## Arguments
- `sigma`: The value of sigma in units of nm.
- `W`: The value of W in units of eV.
- `max_steps`: The maximum number of steps for the simulator.
- `max_steps_minimize`: The maximum number of steps for the minimizer.
- `step_size_minimize`: The step size for the minimizer in units of nm.
- `tol`: The tolerance for convergence in units of kg*m*s^-2.
- `log_stream`: The stream to log the output.

## Returns
- An instance of ABCSimulator.
"""
function ABCSimulator(;
                        sigma=0.1*u"Å", W=1e-2*u"eV", max_steps=100, max_steps_minimize=100, step_size_minimize=0.1u"Å",tol=1e-4u"eV/Å",
                        log_stream=devnull)
    return ABCSimulator(sigma, W, max_steps, max_steps_minimize, step_size_minimize, tol, log_stream)
end

# Penalty function with Gaussuan form
"""
Returns a penalty function of system coordinate x with Gaussuan form
x:      System coordinate
x_0:    Reference system coordinate
sigma:  Spatial extent of the activation, per sqrt(degree of freedom)
W:      Strenth of activation, per degree of freedom
pbc:    Periodic boundary conditions
"""
function f_phi_p(x::Vector{SVector{3, typeof(1.0u"Å")}}, x_0, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"); nopenalty_atoms=[])
    N::Int = length(x)
    E_multiplier = ones(length(x))
    for atom in nopenalty_atoms
        E_multiplier[atom] = 0
    end
    
    sigma2_new = sigma^2
    EDSQ = (A, B) -> sum(sum(map(x -> x.^2, (A-B).*E_multiplier)))
    # phi_p = sum([W * exp(-EDSQ(x,c) / (2*sigma2_new)) for c in x_0]) # unit eV
    phi_p = 0.0u"eV"
    for c in x_0
        if EDSQ(x,c)<9*sigma2_new
            phi_p_individual = W * (exp(-EDSQ(x,c) / (2*sigma2_new)) - exp(-9/2))
            phi_p += phi_p_individual
        end
    end
    return phi_p
end

function grad_f_phi_p(x::Vector{SVector{3, typeof(1.0u"Å")}}, x_0, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"); nopenalty_atoms=[])
    N::Int = length(x)
    E_multiplier = ones(length(x))
    for atom in nopenalty_atoms
        E_multiplier[atom] = 0
    end

    sigma2_new = sigma^2
    EDSQ = (A, B) -> sum(sum(map(x -> x.^2, (A-B).*E_multiplier)))

    grad_phi_p = [(@SVector zeros(Float64,3))*u"eV/Å" for i in 1:N] # unit eV/Å
    for c in x_0
        if EDSQ(x,c)<9*sigma2_new
            grad_phi_p_individual = W * exp(-EDSQ(x,c) / (2*sigma2_new)) / (2*sigma2_new) * 2*(c-x).*E_multiplier
            grad_phi_p += grad_phi_p_individual
        end
    end
    return grad_phi_p
end

# Calculate the gradient of the penalty energy
function penalty_forces(sys::System, penalty_coords, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"); nopenalty_atoms=[])
    # Function of the penalty energy for a given coordinate
    # f_phi_p_coords = x -> f_phi_p(x, penalty_coords, sigma, W)

    # Calculate the gradient of the penalty energy, The penalty force is the negative gradient of the penalty energy
    # penalty_fs = -gradient(f_phi_p_coords, sys.coords)[1] # unit eV/Å
    penalty_fs = -grad_f_phi_p(sys.coords, penalty_coords, sigma, W, nopenalty_atoms=nopenalty_atoms) # unit eV/Å

    return penalty_fs
end

# Define the forces function with penalty term
"""
Evaluate the forces acting on the system with penalty term
If there is no penalty term, the penalty_coords should be set to nothing, 
and return the forces identical to the original forces function
"""
function Molly.forces(sys::System, interaction::EAMInteractionJulia, penalty_coords, sigma::typeof(1.0u"Å"), W::typeof(1.0u"eV"), neighbors_all::Vector{Vector{Int}};
    n_threads::Integer=Threads.nthreads(), nopenalty_atoms=[]) 

    
    fs = interaction.f_forces(interaction.calculator, sys, neighbors_all)

    # Add penalty term to forces
    if penalty_coords != nothing
        fs += penalty_forces(sys, penalty_coords, sigma, W, nopenalty_atoms=nopenalty_atoms) # ev/Å
        # print(maximum(norm.(penalty_forces(sys, penalty_coords, sigma, W))),"\n")
    end
    return fs
end

"""
f_energy_phi(sys::System, sim::Simulator, penalty_coords)

Compute the total energy of the system `sys` including the potential energy contribution from the penalty coordinates.

# Arguments
- `sys::System`: The system for which the energy is to be computed.
- `sim::Simulator`: The simulator object containing simulation parameters.
- `penalty_coords`: The penalty coordinates used to calculate the potential energy contribution.

# Returns
- `E`: The total energy of the system with penalty terms.

"""
function f_energy_phi(sys::System, sim::ABCSimulator, interaction::EAMInteractionJulia, penalty_coords, neighbors_all; nopenalty_atoms=[])
    E_phi = 0*u"eV"
    if penalty_coords!=nothing
        E_phi += f_phi_p(sys.coords, penalty_coords, sim.sigma, sim.W, nopenalty_atoms=nopenalty_atoms)
    end
    E = interaction.f_energy(interaction.calculator, sys, neighbors_all) + E_phi
    return E
end

"""
Minimize_momentum!(sys, sim, penalty_coords; 
                   n_threads::Integer, frozen_atoms=[], neig_inteval::Int=1000, beta::Float64=0.0, print_nsteps=false)

Minimizes the system `sys` energy using the simulatior `sim` providing penalty coordinates `penalty_coords`.

# Arguments
- `sys`: The system to be minimized.
- `sim`: The simulation object.
- `penalty_coords`: The penalty coordinates used in the minimization.
- `n_threads`: The number of threads to use in the minimization.
- `frozen_atoms`: (optional) A list of atoms to be frozen during the minimization.
- `n_threads`: The number of threads to use in the minimization.
- `frozen_atoms`: (optional) A list of atoms to be frozen during the minimization.
- `neig_inteval`: (optional) Inteval between neighbor list updates.
- `beta`: (optional) Momentum coefficient. if beta=0 reduced to gradient descent.
- `print_nsteps`: (optional) Print number of steps required to converge.
"""
function Minimize_momentum!(sys::System, sim::ABCSimulator, interaction::EAMInteractionJulia, penalty_coords, neighbors_all::Vector{Vector{Int}}; 
                            n_threads::Integer=1, frozen_atoms=[], neig_inteval::Int=1000, beta::Float64=0.0, print_nsteps::Bool=false)
    hn = sim.step_size_minimize

    # 1. initialize
    # initialize system energy with penalty
    E = f_energy_phi(sys, sim, interaction, penalty_coords, neighbors_all)
    
    # Set F_multiplier of frozen_atoms to zero
    F_multiplier = ones(length(sys.coords))
    for atom in frozen_atoms
        F_multiplier[atom] = 0
    end

    F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all)
    F = F.*F_multiplier
    max_force_0 = maximum(norm.(F))
    F_copy = copy(F)
    n_accept = 0
    n_reject = 0
    string_convergence="\n"
    for step_n in 1:sim.max_steps_minimize
    # step_n = 0
    # while n_accept < sim.max_steps_minimize
        # step_n+=1
        # Calculate the forces using the new forces function
        # penalty_coords is fixed throughout the minimization

        # 2. evaluate force 
        if step_n % neig_inteval == 0
            neighbors_all = get_neighbors_all(sys)
        end
        F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all)
        F = F.*F_multiplier
        max_force = maximum(norm.(F))

        if step_n > 1
            # 3. check convergence          
            if  max_force < sim.tol
                if print_nsteps
                    print(" Minimization converged after ", step_n, " steps\n")
                end
                break
            end

            # 4. modify force with Momentum
            F = beta*F_copy + (1-beta)*F
            max_force = maximum(norm.(F))
        end

        # 5. update coordinate
        coords_copy = copy(sys.coords)
        coords_update = hn * F ./ max_force # ensure that the maximum move at the each step is sim.step_size_minimize
        sys.coords += coords_update

        # 6. copy neighbor list
        neighbors_all_copy = copy(neighbors_all)
        
        # 7. if energy didn't reduce, revert coordinate back to the previous step.
        # System energy after applying displacements in this step
        if step_n % neig_inteval == 0
            neighbors_all = get_neighbors_all(sys)
        end
        # neighbors_all = get_neighbors_all(sys)
        E_trial = f_energy_phi(sys, sim, interaction, penalty_coords, neighbors_all)
        if E_trial < E
            hn = hn * 6/5
            # hn = min(hn, 1e-1u"Å")
            E = E_trial
            F_copy = copy(F) # copy force for momentum term in the next step
            n_accept += 1
            string_convergence*="1"
        else
            # revert to previous coordinate
            sys.coords = coords_copy
            neighbors_all = neighbors_all_copy
            hn = hn / 5
            # hn = max(hn, 1e-6u"Å")
            # F_copy = copy(F)
            n_reject += 1
            string_convergence*="0"
        end
    end
    
    neighbors_all = get_neighbors_all(sys)
    F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all)
    F = F.*F_multiplier
    max_force = maximum(norm.(F))
    @printf("max force = %e eV/Å ,n_accept: %d, n_reject: %d ",ustrip(max_force), n_accept, n_reject)
    # print(string_convergence,"\n")
    return sys
end


"""
Minimize_FIRE!(sys::System, sim::ABCSimulator, interaction::EAMInteractionJulia, penalty_coords, neighbors_all::Vector{Vector{Int}};
                n_threads::Integer=1, frozen_atoms=[], neig_inteval::Int=1000, print_nsteps=false,
                mass::typeof(1.0u"u")=26.9815u"u",acoef_0::Float64=0.1,alpha::Float64=0.99)

Minimizes the system using the Fast Inertial Relaxation Engine (FIRE) algorithm.

# Arguments
- `sys::System`: The system to be minimized.
- `sim::ABCSimulator`: The ABC simulator.
- `interaction::EAMInteractionJulia`: The EAM interaction.
- `penalty_coords`: The penalty coordinates.
- `neighbors_all::Vector{Vector{Int}}`: The list of neighbor indices for each atom.
- `n_threads::Integer`: The number of threads to use for parallelization. Default is 1.
- `frozen_atoms`: The indices of atoms that are frozen during the minimization. Default is an empty array.
- `neig_inteval::Int`: The interval at which neighbor lists are updated. Default is 1000.
- `print_nsteps`: Whether to print the number of steps during the minimization. Default is false.
- `mass::typeof(1.0u"u")`: The mass of the atoms. Default is 26.9815u"u".
- `acoef_0::Float64`: The initial value of the acceleration coefficient. Default is 0.1.
- `alpha::Float64`: The parameter controlling the step size. Default is 0.99.
"""
function Minimize_FIRE!(sys::System, sim::ABCSimulator, interaction::EAMInteractionJulia, penalty_coords, neighbors_all::Vector{Vector{Int}};
                        n_threads::Integer=1, frozen_atoms=[], neig_inteval::Int=1000, print_nsteps=false,
                        mass::typeof(1.0u"u")=26.9815u"u",acoef_0::Float64=0.1,alpha::Float64=0.99)
    dt_0 = sim.step_size_minimize # time unit (ps)
    dt = dt_0
    acoef = acoef_0
    # Set F_multiplier of frozen_atoms to zero
    F_multiplier = ones(length(sys.coords))
    for atom in frozen_atoms
        F_multiplier[atom] = 0
    end
  
    E = f_energy_phi(sys, sim, interaction, penalty_coords, neighbors_all)

    v_0 = [(@SVector zeros(3))*u"eV/Å/u*ps" for i in 1:length(sys.coords)] # force/mass*time
    v = v_0
    for step_n in 1:sim.max_steps_minimize
        if step_n % neig_inteval == 0
            neighbors_all = get_neighbors_all(sys)
        end

        # 1. skier force
        F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all)
        F = F.*F_multiplier
        
        P = sum(sum([v[i].*F[i] for i in 1:length(v)]))   
        vv = sum(sum([v[i].*v[i] for i in 1:length(v)]))
        FF = sum(sum([F[i].*F[i] for i in 1:length(v)]))
        if ustrip(P)>0.0
            # dt *= 1.1
            dt = min(dt*1.2, 1e3*dt_0)
            v = (1-acoef)*v + acoef*sqrt(vv/FF)*F
            acoef = acoef * alpha
            
        else
            # dt *= 0.5
            dt = max(dt*0.5, 1e-1*dt_0)
            v = v_0
            acoef = acoef_0
        end

        # 2. MD
        accl = F/mass # force/mass
        v += accl.*dt # force/mass*time

        # 3. update coordinate
        coords_update = v .* dt
        sys.coords .+= coords_update # force/mass*time*2

        E_trial = f_energy_phi(sys, sim, interaction, penalty_coords, neighbors_all)
        # print(E,"\n")
        # if E_trial>E
        #     # print("zeroing velocity")
        #     v*=0
        # else
        #     E = E_trial
        # end

    end
    neighbors_all = get_neighbors_all(sys)
    F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all)
    F = F.*F_multiplier
    max_force = maximum(norm.(F))
    @printf("max force = %e eV/Å ",ustrip(max_force))
    return sys
end


"""
Minimize_MD!(sys::System, sim::ABCSimulator, interaction::EAMInteractionJulia, penalty_coords, neighbors_all::Vector{Vector{Int}};
                        n_threads::Integer=1, frozen_atoms=[], neig_inteval::Int=1000, print_nsteps=false,
                        mass::typeof(1.0u"u")=26.9815u"u", acoef_0::Float64=0.1,alpha::Float64=0.99,constrained=false)

Minimizes the molecular dynamics (MD) system using the ABC algorithm.

# Arguments
- `sys::System`: The molecular dynamics system.
- `sim::ABCSimulator`: The ABC simulator.
- `interaction::EAMInteractionJulia`: The EAM interaction.
- `penalty_coords`: The penalty coordinates.
- `neighbors_all::Vector{Vector{Int}}`: The neighbor list.

# Optional Arguments
- `n_threads::Integer=1`: The number of threads to use.
- `frozen_atoms=[]`: The list of frozen atoms.
- `neig_inteval::Int=1000`: The neighbor interval.
- `print_nsteps=false`: Whether to print the number of steps.
- `mass::typeof(1.0u"u")=26.9815u"u"`: The mass of the atoms.
- `acoef_0::Float64=0.1`: The initial value of the a coefficient.
- `alpha::Float64=0.99`: The alpha coefficient.
- `constrained=false`: Whether the system is constrained.

# Returns
- `sys`: The updated molecular dynamics system.

"""
function Minimize_MD!(sys::System, sim::ABCSimulator, interaction::EAMInteractionJulia, 
                      penalty_coords, neighbors_all::Vector{Vector{Int}};
                      n_threads::Integer=1, frozen_atoms=[], nopenalty_atoms=[], neig_inteval::Int=1000, print_nsteps=false,
                      mass::typeof(1.0u"u")=26.9815u"u", acoef_0::Float64=0.1,alpha::Float64=0.99,constrained=false, etol = 1e-4)
    N = length(sys.coords)
    dt_0 = sim.step_size_minimize # time unit (ps)
    dt = dt_0
    acoef = acoef_0
    # Set F_multiplier of frozen_atoms to zero
    F_multiplier = ones(length(sys.coords))
    for atom in frozen_atoms
        F_multiplier[atom] = 0
    end
    
    # energy before move
    E_0 = f_energy_phi(sys, sim, interaction, penalty_coords, neighbors_all, nopenalty_atoms=nopenalty_atoms)
    E = E_0
    deltaE_current = E_0*0
    deltaE_1 = E_0*0
    # print(E,"\n")

    v_0 = [(@SVector zeros(3))*u"eV/Å/u*ps" for i in 1:length(sys.coords)] # force/mass*time
    v = v_0
    for step_n in 1:sim.max_steps_minimize
        if step_n % neig_inteval == 0
            neighbors_all = get_neighbors_all(sys)
        end
        # 1. calculate Force
        F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all, nopenalty_atoms=nopenalty_atoms)
        F = F.*F_multiplier
        # max_force = maximum(norm.(F))
        # print(max_force,"\n")

        ## update stepsize and velocity like FIRE algorithm
        # P = sum(sum([v[i].*F[i] for i in 1:length(v)]))   
        # vv = sum(sum([v[i].*v[i] for i in 1:length(v)]))
        # FF = sum(sum([F[i].*F[i] for i in 1:length(v)]))
        # if ustrip(P)>0.0
        #     # dt *= 1.1
        #     # dt = min(dt*1.2, 1e3*dt_0)
        #     v = (1-acoef)*v + acoef*sqrt(vv/FF)*F
        #     acoef = acoef * alpha
            
        # else
        #     # dt *= 0.5
        #     # dt = max(dt*0.5, 1e-1*dt_0)
        #     acoef = acoef_0
        # end

        # 2. MD
        accl = F/mass # force/mass
        v += accl.*dt # force/mass*time
        coords_update = v .* dt # force/mass*time^2
        # if constrained
        #     coords_update[N] = @SVector [coords_update[N][1], 0.0u"Å", coords_update[N][3]]
        # end
        sys.coords .+= coords_update # force/mass*time*2

        # energy after move
        E_trial = f_energy_phi(sys, sim, interaction, penalty_coords, neighbors_all, nopenalty_atoms=nopenalty_atoms)
        if step_n == 1
        # terminate condition
        else 
            deltaE_current = abs(E_trial-E)
            deltaE_1 = abs(E_trial-E_0)
            if deltaE_current<etol*deltaE_1
                break
            end
        end

        # print(E,"\n")
        if E_trial>E
            # print("zeroing velocity")
            v = v_0
            dt = max(dt*0.9, 3e-1*dt_0)
        else
            E = E_trial
            dt = min(dt*1.1, 3*dt_0)
        end

    end
    neighbors_all = get_neighbors_all(sys)
    F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all, nopenalty_atoms=nopenalty_atoms)
    F = F.*F_multiplier
    max_force = maximum(norm.(F))
    # @printf("max force = %e eV/Å ",ustrip(max_force))
    @printf("deltaE_current/deltaE_1 = %e ", deltaE_current/deltaE_1)
    return sys
end


# Implement the simulate! function for ABCSimulator
"""
simulate!(sys, sim::ABCSimulator; n_threads::Integer=Threads.nthreads(), frozen_atoms=[], run_loggers=true, fname="output.txt")

Simulates the system using the ABCSimulator.

# Arguments
- `sys`: The system to be simulated.
- `sim`: An instance of the ABCSimulator.
- `n_threads`: The number of threads to use for parallel execution. Defaults to the number of available threads.
- `frozen_atoms`: A list of atoms that should be frozen during the simulation.
- `run_loggers`: A boolean indicating whether to run the loggers during the simulation.
- `fname`: The name of the output file.

# Examples 
simulate!(molly_system, simulator, n_threads=1, fname="output_test.txt", frozen_atoms=frozen_atoms)
"""
function simulate!(sys::System, sim::ABCSimulator, interaction::EAMInteractionJulia; 
                   n_threads::Integer=Threads.nthreads(), run_loggers::Bool=true, fname::String="output.txt", neig_inteval::Int=1, minimize_only::Bool=false, 
                   d_boost=1.0e-2u"Å", beta=0.0, frozen_atoms=[], nopenalty_atoms=[])
    neighbors_all = get_neighbors_all(sys)
    neighbors = find_neighbors(sys, sys.neighbor_finder; n_threads=n_threads)

    # open an empty output file
    open(fname, "w") do file
        write(file, "")
    end

    # Set d_multiplier of frozen_atoms to zero
    d_multiplier = ones(length(sys.coords))
    for atom in frozen_atoms
        d_multiplier[atom] = 0
    end

    # 0. Call Minimize! without penalty_coords before the loop
    # Minimize_momentum!(sys, sim, interaction, nothing, neighbors_all; n_threads=n_threads, frozen_atoms=frozen_atoms, neig_inteval=neig_inteval, beta=beta, print_nsteps=true)
    Minimize_MD!(sys, sim, interaction, nothing, neighbors_all; n_threads=n_threads, frozen_atoms=frozen_atoms, neig_inteval=neig_inteval, mass=100.0u"u", nopenalty_atoms=nopenalty_atoms)
    E = interaction.f_energy(interaction.calculator, sys, neighbors_all)

    if minimize_only
        return sys
    end

    # Run the loggers, Log the step number (or other details as needed)
    run_loggers!(sys, neighbors, 0, run_loggers; n_threads=n_threads)
    @printf("step %d: ",0)
    print(E)
    print("\n")

    ## 1. Store the initial coordinates
    penalty_coords = [copy(sys.coords)]  

    F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all, nopenalty_atoms=nopenalty_atoms)
    F = F.*d_multiplier
    max_force_0 = maximum(norm.(F))
    for step_n in 1:sim.max_steps
        ## 2. Slightly perturb the system coordinates
        for i in 1:length(sys.coords)
            random_direction = randn(size(sys.coords[i]))
            sys.coords[i] += d_boost * random_direction*d_multiplier[i]
        end

        neighbors_all = get_neighbors_all(sys)

        ## 3. Call Minimize! with penalty_coords, update system coordinates
        ## energy before minimization
        coords_before = copy(sys.coords)
        E_before = f_energy_phi(sys, sim, interaction, penalty_coords, neighbors_all, nopenalty_atoms=nopenalty_atoms)

        ## run the minimization algorithm
        # Minimize_momentum!(sys, sim, interaction, penalty_coords, neighbors_all; n_threads=n_threads, frozen_atoms=frozen_atoms, neig_inteval=neig_inteval, beta=beta, print_nsteps=true)
        Minimize_MD!(sys, sim, interaction, penalty_coords, neighbors_all; n_threads=n_threads, frozen_atoms=frozen_atoms, neig_inteval=neig_inteval, mass=100.0u"u", nopenalty_atoms=nopenalty_atoms)
        E = interaction.f_energy(interaction.calculator, sys, neighbors_all)
        
        @printf("step %d: ",step_n)
        print(E)
        print("\n")

        if step_n % 1==0
            # Run the loggers, Log the step number (or other details as needed)
            run_loggers!(sys, neighbors, step_n, run_loggers; n_threads=n_threads)
            # println(sim.log_stream, "Step 0 - potential energy ",
            #         E, " - max force N/A - N/A")
        end

        ## energy after minimization
        # neighbors_all = get_neighbors_all(sys)
        E_phi = f_phi_p(sys.coords, penalty_coords, sim.sigma, sim.W, nopenalty_atoms=nopenalty_atoms)
        E_after = E+E_phi
        open(fname, "a") do file
            write(file, string(ustrip(E))*" "*string(ustrip(E_phi))*"\n")
        end

        ## 4. Update penalty_coords for the next step
        neighbors_all = get_neighbors_all(sys)
        F = forces(sys, interaction, penalty_coords, sim.sigma, sim.W, neighbors_all, nopenalty_atoms=nopenalty_atoms)
        F = F.*d_multiplier

        max_force = maximum(norm.(F))
        # if max_force<sim.tol
        push!(penalty_coords, copy(sys.coords))
        # end
        # if E_after>E_before
        #     sys.coords = coords_before
        #     # d_multiplier*=-1
        # end
    end
    return sys
end


simulate!

### Initialize System and run simulator

In [7]:
eamJulia = EAMInteractionJulia(eam,calculate_energy,calculate_forces)
function initialize_system(loggers=(coords=CoordinateLogger(1),))
    molly_atoms, atoms_ab, box_size, atom_positions, _ = system_adatom((5,5,6))

    # Specify boundary condition
    boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])

    atom_positions_init = copy(atom_positions)
    molly_atoms_init = copy(molly_atoms)
    # Initialize the system with the initial positions and velocities
    system_init = Molly.System(
    atoms=molly_atoms_init,
    atoms_data = [AtomData(element="Al") for a in molly_atoms_init],
    coords=atom_positions_init,  # Ensure these are SVector with correct units
    boundary=boundary_condition,
    # loggers=Dict(:kinetic_eng => Molly.KineticEnergyLogger(100), :pot_eng => Molly.PotentialEnergyLogger(100)),
    neighbor_finder = DistanceNeighborFinder(
    eligible=trues(length(molly_atoms_init), length(molly_atoms_init)),
    n_steps=1e3,
    dist_cutoff=10u"Å"),
    loggers=loggers,
    energy_units=u"eV",  # Ensure these units are correctly specified
    force_units=u"eV/Å"  # Ensure these units are correctly specified
    )
    return system_init
end

initialize_system (generic function with 2 methods)

In [8]:
molly_system = initialize_system()

neighbors_all = get_neighbors_all(molly_system)
F_multiplier = ones(length(molly_system.coords))

fs = eamJulia.f_forces(eamJulia.calculator, molly_system, neighbors_all)
max_force_before = maximum(norm.(fs.*F_multiplier))
energy_before = eamJulia.f_energy(eamJulia.calculator, molly_system, neighbors_all)

# simulator = ABCSimulator(sigma=1.0*u"Å", W=1.0*u"eV", max_steps=1, max_steps_minimize=1000, step_size_minimize=2e-2u"Å", tol=1e-6u"eV/Å")
# Minimize_momentum!(molly_system, simulator, eamJulia, nothing, neighbors_all; 
#          n_threads=1, frozen_atoms=[],neig_inteval=50, print_nsteps=true, beta=0.5)
simulator = ABCSimulator(sigma=1.0*u"Å", W=1.0*u"eV", max_steps=1, max_steps_minimize=120, step_size_minimize=5e-3u"ps", tol=1e-6u"eV/Å")
Minimize_FIRE!(molly_system, simulator, eamJulia, nothing, neighbors_all;
         n_threads=1, frozen_atoms=[], neig_inteval=5, print_nsteps=true,
         mass=26.9815u"u")

z_coords = [coords[3] for coords in molly_system.coords]
frozen_atoms = [index for (index, z_coord) in enumerate(z_coords) if z_coord < 4.9*al_LatConst*10u"Å"]
print("\n\nfreeze ",length(frozen_atoms)," atoms\n")
F_multiplier = ones(length(molly_system.coords))
for atom in frozen_atoms
    F_multiplier[atom] = 0
end

coord_ad = molly_system.coords[length(molly_system.coords)]
r_focus = [sqrt(sum((coord-coord_ad).^2)) for coord in molly_system.coords]
nopenalty_atoms = [index for (index, r_focus_i) in enumerate(r_focus) if r_focus_i > 0.75*al_LatConst*10u"Å"]
# nopenalty_atoms = [index for (index, z_coord) in enumerate(z_coords) if z_coord < 6.5*al_LatConst*10u"Å"]
print("exclude ",length(nopenalty_atoms)," atoms from E_phi calculation\n")

# Minimize_momentum!(molly_system, simulator, eamJulia, nothing, neighbors_all; 
#          n_threads=1, frozen_atoms=frozen_atoms,neig_inteval=50, print_nsteps=true, beta=0.5)
Minimize_FIRE!(molly_system, simulator, eamJulia, nothing, neighbors_all;
         n_threads=1, frozen_atoms=frozen_atoms, neig_inteval=5, print_nsteps=true,
         mass=26.9815u"u")

neighbors_all = get_neighbors_all(molly_system)
fs = eamJulia.f_forces(eamJulia.calculator, molly_system, neighbors_all)
max_force_after = maximum(norm.(fs.*F_multiplier))
energy_after = eamJulia.f_energy(eamJulia.calculator, molly_system, neighbors_all)
@printf("\n\nmax force before minimization = %e eV/Å\n", ustrip(max_force_before))
@printf("max force after minimization = %e eV/Å\n", ustrip(max_force_after))
@printf("\nmax energy before minimization = %f eV/Å\n", ustrip(energy_before))
@printf("max energy after minimization = %f eV/Å\n", ustrip(energy_after))

max force = 3.912180e-08 eV/Å 

freeze 100 atoms
exclude 296 atoms from E_phi calculation
max force = 4.332151e-13 eV/Å 

max force before minimization = 3.621143e-01 eV/Å
max force after minimization = 4.332151e-13 eV/Å

max energy before minimization = -962.392081 eV/Å
max energy after minimization = -962.518506 eV/Å


In [9]:
N_free = length(molly_system.coords)-length(nopenalty_atoms)
sigma = sqrt(0.01*3*N_free)
W = 0.1
@printf("sigma^2 = %e, %e Å/dof^1/2\n W = %e eV\n",ustrip(sigma^2), ustrip(sigma/sqrt(3*N_free)),ustrip(W))

# simulator = ABCSimulator(sigma=sigma*u"Å", W=W*u"eV", 
#                          max_steps=1000, max_steps_minimize=500, step_size_minimize=5e-2u"Å", tol=1e-5u"eV/Å")
simulator = ABCSimulator(sigma=sigma*u"Å", W=W*u"eV", 
                         max_steps=1000, max_steps_minimize=500, step_size_minimize=2e-3u"ps", tol=1e-3u"eV/Å")
# using ProfileView
# ProfileView.@profview 
simulate!(molly_system, simulator, eamJulia, n_threads=1, fname="output_test_Julia_556.txt",
          neig_inteval=5, minimize_only=false, 
          d_boost=1e-6u"Å", frozen_atoms=frozen_atoms, nopenalty_atoms=nopenalty_atoms)

sigma^2 = 1.500000e-01, 1.000000e-01 Å/dof^1/2
 W = 1.000000e-01 eV
deltaE_current/deltaE_1 = 1.000000e+00 step 0: -962.5185064838313 eV
deltaE_current/deltaE_1 = 0.000000e+00 step 1: -962.5185064838281 eV
deltaE_current/deltaE_1 = 0.000000e+00 step 2: -962.5185064837997 eV
deltaE_current/deltaE_1 = 9.171788e-05 step 3: -962.5185064837592 eV
deltaE_current/deltaE_1 = 0.000000e+00 step 4: -962.5185064836196 eV
deltaE_current/deltaE_1 = 7.783067e-05 step 5: -962.3735985878941 eV
deltaE_current/deltaE_1 = 6.192927e-05 step 6: -962.299589115143 eV
deltaE_current/deltaE_1 = 6.747104e-05 step 7: -962.2947183686412 eV
deltaE_current/deltaE_1 = 2.200803e-05 step 8: -962.3264023129738 eV
deltaE_current/deltaE_1 = 6.752039e-05 step 9: -962.2647317604145 eV
deltaE_current/deltaE_1 = 8.679541e-05 step 10: -962.2990125794845 eV
deltaE_current/deltaE_1 = 7.993703e-06 step 11: -962.3218219910239 eV
deltaE_current/deltaE_1 = 7.610512e-05 step 12: -962.3321940562358 eV
deltaE_current/deltaE_1 = 3.90467

System with 301 atoms, boundary CubicBoundary{Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}}(Quantity{Float64, 𝐋, Unitful.FreeUnits{(Å,), 𝐋, nothing}}[20.227252500000002 Å, 20.2677475 Å, 42.51975 Å])

In [10]:
function visualize_wrap(coord_logger,
                    boundary,
                    out_filepath::AbstractString,
                    fig;
                    connections=Tuple{Int, Int}[],
                    connection_frames=[trues(length(connections)) for i in values(coord_logger)],
                    trails::Integer=0,
                    framerate::Integer=30,
                    color=:purple,
                    connection_color=:orange,
                    markersize=0.05,
                    linewidth=2.0,
                    transparency=true,
                    show_boundary::Bool=true,
                    boundary_linewidth=2.0,
                    boundary_color=:black,
                    az=1.275pi,
                    el=pi/8,
                    kwargs...)
    coords_start = first(values(coord_logger))
    dist_unit = unit(first(first(coords_start)))
    dims = n_dimensions(boundary)
    # fig = Figure()

    if dims == 3
        PointType = Point3f
        ax = Axis3(fig[1, 1], aspect=:data, azimuth=az, elevation=el)
        max_connection_dist = cbrt(box_volume(boundary)) / 2
    elseif dims == 2
        PointType = Point2f
        ax = Axis(fig[1, 1])
        ax.aspect = DataAspect()
        max_connection_dist = sqrt(box_volume(boundary)) / 2
    else
        throw(ArgumentError("found $dims dimensions but can only visualize 2 or 3 dimensions"))
    end

    positions = Observable(PointType.(ustrip_vec.(coords_start)))
    # scatter!(ax, positions; color=color, markersize=markersize, transparency=transparency,
    #             markerspace=:data, kwargs...)
    meshscatter!(ax, positions; color=color, markersize=markersize, transparency=transparency,
    kwargs...)

    if show_boundary
        lines!(
            ax,
            Molly.bounding_box_lines(boundary, dist_unit)...;
            color=boundary_color,
            linewidth=boundary_linewidth,
        )
    end

    connection_nodes = []
    for (ci, (i, j)) in enumerate(connections)
        # Don't display connected atoms that are likely connected over the box edge
        if first(connection_frames)[ci] && norm(coords_start[i] - coords_start[j]) < max_connection_dist
            if dims == 3
                push!(connection_nodes, Observable(PointType.(
                        ustrip.([coords_start[i][1], coords_start[j][1]]),
                        ustrip.([coords_start[i][2], coords_start[j][2]]),
                        ustrip.([coords_start[i][3], coords_start[j][3]]))))
            elseif dims == 2
                push!(connection_nodes, Observable(PointType.(
                        ustrip.([coords_start[i][1], coords_start[j][1]]),
                        ustrip.([coords_start[i][2], coords_start[j][2]]))))
            end
        else
            if dims == 3
                push!(connection_nodes, Observable(PointType.([0.0, 0.0], [0.0, 0.0],
                                                        [0.0, 0.0])))
            elseif dims == 2
                push!(connection_nodes, Observable(PointType.([0.0, 0.0], [0.0, 0.0])))
            end
        end
    end
    for (ci, cn) in enumerate(connection_nodes)
        lines!(ax, cn;
                color=isa(connection_color, AbstractArray) ? connection_color[ci] : connection_color,
                linewidth=isa(linewidth, AbstractArray) ? linewidth[ci] : linewidth,
                transparency=transparency)
    end

    trail_positions = []
    for trail_i in 1:trails
        push!(trail_positions, Observable(PointType.(ustrip_vec.(coords_start))))
        col = parse.(Colorant, color)
        alpha = 1 - (trail_i / (trails + 1))
        alpha_col = RGBA.(red.(col), green.(col), blue.(col), alpha)
        # scatter!(ax, trail_positions[end]; color=alpha_col,  markersize=markersize,
        #             transparency=transparency, markerspace=:data, kwargs...)
        meshscatter!(ax, trail_positions[end]; color=alpha_col, markersize=markersize, transparency=transparency,
                    kwargs...)
    end

    boundary_conv = ustrip.(dist_unit, Molly.cubic_bounding_box(boundary))
    xlims!(ax, Molly.axis_limits(boundary_conv, coord_logger, 1))
    ylims!(ax, Molly.axis_limits(boundary_conv, coord_logger, 2))
    dims == 3 && zlims!(ax, Molly.axis_limits(boundary_conv, coord_logger, 3))

    GLMakie.record(fig, out_filepath, eachindex(values(coord_logger)); framerate=framerate) do frame_i
        coords = values(coord_logger)[frame_i]
        coords = wrap_coords.(coords, (boundary,))
        u_coords = unit(coords[1][1])
        coords = [ustrip(c) for c in coords]*uconvert(u"nm", 1*u_coords)
        # print(coords[1][1])

        for (ci, (i, j)) in enumerate(connections)
            if connection_frames[frame_i][ci] && norm(coords[i] - coords[j]) < max_connection_dist
                if dims == 3
                    connection_nodes[ci][] = PointType.(
                                ustrip.([coords[i][1], coords[j][1]]),
                                ustrip.([coords[i][2], coords[j][2]]),
                                ustrip.([coords[i][3], coords[j][3]]))
                elseif dims == 2
                    connection_nodes[ci][] = PointType.(
                                ustrip.([coords[i][1], coords[j][1]]),
                                ustrip.([coords[i][2], coords[j][2]]))
                end
            else
                if dims == 3
                    connection_nodes[ci][] = PointType.([0.0, 0.0], [0.0, 0.0],
                                                        [0.0, 0.0])
                elseif dims == 2
                    connection_nodes[ci][] = PointType.([0.0, 0.0], [0.0, 0.0])
                end
            end
        end

        positions[] = PointType.(ustrip_vec.(coords))
        for (trail_i, trail_position) in enumerate(trail_positions)
            trail_position[] = PointType.(ustrip_vec.(values(coord_logger)[max(frame_i - trail_i, 1)]))
        end
    end
end


visualize_wrap (generic function with 1 method)

In [11]:
## visualize
color_0 = :dimgray
color_1 = :red
colors = []
for (index, value) in enumerate(molly_system.coords)
    push!(colors, index < length(molly_system.coords) ? color_0 : color_1)
end
framerate = 60
molly_atoms, atoms_ab, box_size, atom_positions = system_adatom((5,5,6))

# Specify boundary condition
boundary_condition = Molly.CubicBoundary(box_size[1],box_size[2],box_size[3])
fig = Figure(size = (800, 800))
visualize_wrap(molly_system.loggers.coords, boundary_condition, "test_Julia_556_top.mp4", fig; 
                markersize=0.1, color=colors, az=-pi/2, el=pi/2, framerate=framerate, transparency=false)
fig = Figure(size = (800, 800))
visualize_wrap(molly_system.loggers.coords, boundary_condition, "test_Julia_556_side.mp4", fig; 
                markersize=0.1, color=colors, az=0, el=0, framerate=framerate, transparency=false)
fig = Figure(size = (800, 800))
visualize_wrap(molly_system.loggers.coords, boundary_condition, "test_Julia_556_side2.mp4", fig; 
                markersize=0.1, color=colors, az=-pi/2, el=0, framerate=framerate, transparency=false)
fig = Figure(size = (800, 800))
visualize_wrap(molly_system.loggers.coords, boundary_condition, "test_Julia_556.mp4", fig; 
                markersize=0.1, color=colors, az=-5*pi/12, framerate=framerate, transparency=false)

"test_Julia_556.mp4"

In [12]:
using JLD2
save_object("./test_Julia_556.jld2", molly_system)
# molly_system_loaded = load_object("./test_Julia_776.jld2")

In [13]:
# molly_system = initialize_system()
# z_coords = [coords[3] for coords in molly_system.coords]
# frozen_atoms = [index for (index, z_coord) in enumerate(z_coords) if z_coord < 4.9*al_LatConst*10u"Å"]
# print("\n\nfreeze ",length(frozen_atoms)," atoms\n")
# F_multiplier = ones(length(molly_system.coords))
# for atom in frozen_atoms
#     F_multiplier[atom] = 0
# end

# neighbors_all = get_neighbors_all(molly_system)
# fs = eamJulia.f_forces(eamJulia.calculator, molly_system, neighbors_all)
# max_force_before = maximum(norm.(fs.*F_multiplier))
# energy_before = eamJulia.f_energy(eamJulia.calculator, molly_system, neighbors_all)

# simulator = ABCSimulator(sigma=1.0*u"Å", W=1.0*u"eV", max_steps=1, max_steps_minimize=400, step_size_minimize=2e-2u"ps", tol=1e-6u"eV/Å")
# Minimize_FIRE!(molly_system, simulator, eamJulia, nothing, neighbors_all;
#                n_threads=1, frozen_atoms=frozen_atoms, neig_inteval=200, print_nsteps=false,
#                mass=26.9815u"u",energy_inteval=1)

# neighbors_all = get_neighbors_all(molly_system)
# fs = eamJulia.f_forces(eamJulia.calculator, molly_system, neighbors_all)
# max_force_after = maximum(norm.(fs.*F_multiplier))
# energy_after = eamJulia.f_energy(eamJulia.calculator, molly_system, neighbors_all)

# @printf("\n\nmax force before minimization = %e eV/Å\n", ustrip(max_force_before))
# @printf("max force after minimization = %e eV/Å\n", ustrip(max_force_after))
# @printf("\nmax energy before minimization = %f eV/Å\n", ustrip(energy_before))
# @printf("max energy after minimization = %f eV/Å\n", ustrip(energy_after))